In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [37]:
from striprtf.striprtf import rtf_to_text
import json

# Read the .rtf file content
with open("algoparams_from_ui.json.rtf", "r") as file:
    rtf_content = file.read()

In [38]:
# Convert RTF to plain text
plain_text = rtf_to_text(rtf_content)

# Now try to load the plain text as JSON
try:
    config = json.loads(plain_text)
    print("JSON successfully loaded.")
except json.JSONDecodeError as e:
    print(f"Failed to decode JSON: {e}")

JSON successfully loaded.


In [39]:
# Load the dataset
df = pd.read_csv(config['design_state_data']['session_info']['dataset'])

In [40]:
for feature, details in config['design_state_data']['feature_handling'].items():
    if details['is_selected']:
        feature_details = details['feature_details']
        if 'missing_values' in feature_details and feature_details['missing_values'] == "Impute":
            strategy = "mean" if feature_details.get('impute_with') == "Average of values" else "constant"
            fill_value = feature_details.get('impute_value', 0)
            # Your logic for handling the imputation


In [41]:
# Feature Generation: Add interaction features
for interaction in config['design_state_data']['feature_generation']['linear_interactions']:
    feature_1, feature_2 = interaction
    df[f"{feature_1}_{feature_2}_interaction"] = df[feature_1] * df[feature_2]



In [42]:
# Separate features and target variable
target_column = config['design_state_data']['target']['target']
X = df.drop(columns=[target_column])
y = df[target_column]

In [43]:
# Identify and encode categorical features
categorical_features = X.select_dtypes(include=['object']).columns
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "RandomForestRegressor": RandomForestRegressor(),
    # Add other models as necessary
}

# Model selection based on JSON configuration
selected_model_name = None
for model_name, model_info in config['design_state_data']['algorithms'].items():
    if model_info['is_selected']:
        selected_model_name = model_name
        break

if selected_model_name and selected_model_name in models:
    selected_model = models[selected_model_name]

    # Extract hyperparameters from JSON configuration
    param_grid = {
        'n_estimators': [int(config['design_state_data']['hyperparameters'].get('num_of_trees', 100))]
    }

    # Handle max_depth separately if it's not None
    depth_of_trees = config['design_state_data']['hyperparameters'].get('depth_of_trees', None)
    if depth_of_trees is not None:
        param_grid['max_depth'] = [int(depth_of_trees)]

    # Print debug information
    print(f"Selected Model: {selected_model}")
    print(f"Model parameters for GridSearch: {param_grid}")

    # Grid Search
    grid_search = GridSearchCV(selected_model, param_grid,
                               cv=int(config['design_state_data']['hyperparameters']['num_of_folds']),
                               n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Predictions
    y_pred = grid_search.predict(X_test)

    # Log metrics
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
    print(f"R2 Score: {r2_score(y_test, y_pred)}")

else:
    print("No model selected or invalid model name in the JSON configuration.")


Selected Model: RandomForestRegressor()
Model parameters for GridSearch: {'n_estimators': [100]}
Best parameters: {'n_estimators': 100}
Mean Squared Error: 0.032685405666666556
R2 Score: 0.9485800047195373
